In [ ]:
import importlib
from termcolor import colored
from utils import io_helpers
from utils import llm

importlib.reload(io_helpers)

documents = io_helpers.get_documents(read_relations=True)

prompts = io_helpers.get_prompt(name="comparison/check_for_contradictions")
system_prompt = prompts["system_prompt"]

initial_doc_pairs = [
    (100134, 100139, False),
    (100134, 134, True),
    (100134, 127, False),
    (100134, 70, False),
]


for id1, id2, expected_result in initial_doc_pairs:
    text1 = documents[documents["doc_id"].astype(int) == int(id1)].squeeze()["content"]
    text2 = documents[documents["doc_id"].astype(int) == int(id2)].squeeze()["content"]

    user_prompt = llm.format_user_prompt(prompts["user_prompt"], text1=text1, text2=text2)
    actual_result = llm.call_openai(system_prompt, user_prompt).contradictory_info_found

    print(f"-- {id1} & {id2} --")
    print(f"expected result: {expected_result}")
    print(f"actual result: {actual_result}")
    if expected_result == actual_result:
        print(colored("Check!", "green"))
    else:
        print(colored("Wrong!", "red"))

-- 100134 & 100139 --
expected result: False
actual result: False
Check!
-- 100134 & 134 --
expected result: True
actual result: False
Wrong!
-- 100134 & 127 --
expected result: False
actual result: False
Check!
-- 100134 & 70 --
expected result: False
actual result: False
Check!


<class 'numpy.int64'>
